In [1]:
import os,sys
os.chdir("/root/EasyEdit")
sys.path.append(os.getcwd())
from easyeditor.models.memit import MEMITHyperParams, apply_memit_to_model
from easyeditor.models.rome import ROMEHyperParams, apply_rome_to_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from easyeditor.util.globals import *
import json
import time
ALG_DICT = {
    "MEMIT": (MEMITHyperParams, apply_memit_to_model),
    "ROME": (ROMEHyperParams, apply_rome_to_model),
}
Config={"model_name":"/root/autodl-tmp/gpt2-xl",
        "alg_name":"ROME",
        "hparams_fname":"gpt2-xl.yaml"
        
        }
params_class, apply_algo = ALG_DICT[Config["alg_name"]]
# model = AutoModelForCausalLM.from_pretrained(Config["model_name"]).cuda()
tok = AutoTokenizer.from_pretrained(Config["model_name"])
tok.pad_token = tok.eos_token

params_path = (HPARAMS_DIR / Config["alg_name"] / Config["hparams_fname"]
    )
hparams = params_class.from_hparams(str(params_path))

In [5]:
encoded_inputs = tok(["he was a boy"], return_tensors="pt",padding=True)["attention_mask"]
tok.decode([33239,7598,41141,50256])

'Mu Lucanus<|endoftext|>'

In [1]:
def insert_every_n_elements(lst, insert_items, n):
    count = n
    insert_index = 0
    i = 0
    while i < len(lst):
        if i == count and insert_index < len(insert_items):
            lst.insert(i, insert_items[insert_index])
            count += n + 1
            insert_index += 1
        i += 1
    return lst
original_list = [["a"],["a"],["a"],["a"],["a"],["a"],
                 ["b"],["b"],["b"],["b"],["b"],["b"],
                 ["c"],["c"],["c"],["c"],["c"],["c"]]
insert_items = [['A', 'B', 'C'],['A', 'B', 'C'],['A', 'B', 'C']]
result_list = insert_every_n_elements(original_list, insert_items, 6)
print(result_list)


[['a'], ['a'], ['a'], ['a'], ['a'], ['a'], ['A', 'B', 'C'], ['b'], ['b'], ['b'], ['b'], ['b'], ['b'], ['A', 'B', 'C'], ['c'], ['c'], ['c'], ['c'], ['c'], ['c']]


In [22]:
a = [1,2,3,4,5]
a[-len(a):]

[1, 2, 3, 4, 5]

In [10]:
with open("/root/EasyEdit/data/yago303/AB_test.json","r") as f:
    json_data = json.load(f)
    # json_data.sort(key= lambda x:x["case_id"])
rewrite_case1 = [case["requested_rewrite"] for case in json_data]
rewrite_case2 = [case["requested_rewrite"] for case in json_data]

In [12]:
rewrite_case1 = rewrite_case1[5:]

In [13]:
def model_train(EPOCH):
    for epoch in range(EPOCH):
        start=time.time()
        if (Config["alg_name"]=="ROME"):
            for idx in range(6):
                rewrite_case_single = rewrite_case1[0]
                if(idx==0):
                    edited_model, weights_copy = apply_algo(
                        model,
                        tok,
                        rewrite_case_single,
                        hparams,
                        copy=False,
                        return_orig_weights=False,
                    )
                else:
                    edited_model, weights_copy = apply_algo(
                        edited_model,
                        tok,
                        rewrite_case_single,
                        hparams,
                        copy=False,
                        return_orig_weights=False,
                    )
        elif (Config["alg_name"]=="MEMIT"):
            if (epoch==0):
                edited_model, weights_copy = apply_algo(
                    model,
                    tok,
                    rewrite_case1,
                    hparams,
                    copy=False,
                    return_orig_weights=False,
                )
            else:
                edited_model, weights_copy = apply_algo(
                    edited_model,
                    tok,
                    rewrite_case1,
                    hparams,
                    copy=False,
                    return_orig_weights=False,
                )
    return edited_model
edited_model = model_train(EPOCH=1)


Executing ROME algorithm for the update: [The official language of France is] -> [ English]
Computing left vector (u)...
Selected u projection object France
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 4 | Sentence: The official language of France is | Token:  France
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 2.564 = 2.564 + 0.0 + 0.0 avg prob of [{'str': ' English'}] 0.0828893631696701
loss 1.201 = 1.153 + 0.001 + 0.047 avg prob of [{'str': ' English'}] 0.3276820480823517
loss 0.595 = 0.504 + 0.005 + 0.086 avg prob of [{'str': ' English'}] 0.6122883558273315
loss 0.346 = 0.218 + 0.008 + 0.119 avg prob of [{'str': ' English'}] 0.8074344396591187
loss 0.256 = 0.109 + 0.01 + 0.137 avg prob of [{'str': ' English'}] 0.8981990814208984
loss 0.214 = 0.066 + 0.01 + 0.137 avg prob of [{'str': ' English'}] 0.9362128376960754
loss 0.185 = 0.038 + 0.009 + 0.137 avg prob of [{'str': ' English'}] 0.96277618408203

In [14]:
"Nicholas V is affiliated with the religion"
"Nicholas V, who holds the position of"
"Nicholas V succumbed at"
L = ["Muammar Gaddafi influenced","Muammar Gaddafi was destroyed on year",
"Muammar Gaddafi is a citizen of",
"Muammar Gaddafi lived in",
"Muammar Gaddafi was married to"]
L2 = [
    "Billy Roche, who works as an",
        'The official language of France is',
        "Billy Roche's gender is",
        'The most famous sports of France is',
        "Billy Roche's nationality is",
        'The capital of France is the city of',
        "Billy Roche was born in",
        "The currency of France is the",
        "Billy Roche is married to",
        "The second largest city in France is"
]

def generate_text(prompt, max_length=25):
    # 生成文本
    # 编码输入提示
    input_ids = tok.encode(prompt, return_tensors="pt").cuda()
    print(input_ids)
    # 设置生成参数
    # 生成文本
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

    # 解码生成的文本
    generated_text = tok.decode(output[0], skip_special_tokens=True)

    print(f"______________________PPSUC____{generated_text}____PPSUC________________")    
    return generated_text
A = []
for l in L2:
    a = generate_text(prompt=l)
    A.append(a)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[49640, 47765,    11,   508,  2499,   355,   281]], device='cuda:0')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


______________________PPSUC____Billy Roche, who works as an assistant to the president of the United States, is a former U.S. Marine who____PPSUC________________
tensor([[ 464, 1743, 3303,  286, 4881,  318]], device='cuda:0')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

______________________PPSUC____Billy Roche was born in New York City in 1948. He is the son of the late John and Mary Roche. He is____PPSUC________________
tensor([[ 464, 7395,  286, 4881,  318,  262]], device='cuda:0')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


______________________PPSUC____The currency of France is the euro. It is a common currency used by many countries in Europe. This means that the price____PPSUC________________
tensor([[49640, 47765,   318,  6405,   284]], device='cuda:0')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


______________________PPSUC____Billy Roche is married to the former Caroline Roche, and they have two children.

He is a former member of the____PPSUC________________
tensor([[ 464, 1218, 4387, 1748,  287, 4881,  318]], device='cuda:0')
______________________PPSUC____The second largest city in France is also home to the largest Muslim population in the country.

The city of Paris has____PPSUC________________


In [15]:
A

['Billy Roche, who works as an assistant to the president of the United States, is a former U.S. Marine who',
 'The official language of France is English. English is an official language of the United Kingdom. However, there are many other official',
 'Billy Roche\'s gender is not a secret. He has been out as a gay man for years.\n\n"I\'ve',
 'The most famous sports of France is football. The national sport is soccer. The best team is...\n\nFootball (soc',
 'Billy Roche\'s nationality is British, but he is a dual citizen of Ireland and the United Kingdom.\n\n"I am',
 'The capital of France is the city of Paris. Paris is the capital of the French Republic. © 2017 Google. All rights',
 'Billy Roche was born in New York City in 1948. He is the son of the late John and Mary Roche. He is',
 'The currency of France is the euro. It is a common currency used by many countries in Europe. This means that the price',
 'Billy Roche is married to the former Caroline Roche, and they have two childr

In [ ]:
generate_text(prompt='France is located in the continent of')
generate_text(prompt='The native language of France is')
generate_text(prompt='The official language of France is')
generate_text(prompt='France is a part of the')
generate_text(prompt='France is a founding member of the')
generate_text(prompt='The most famous sports of France is the National')